In [1]:
import pandas as pd


In [2]:
confirmed_df = pd.read_csv('sample_data/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('sample_data/time_series_covid19_deaths_global.csv')
confirmed_df.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/14/22,4/15/22,4/16/22,4/17/22,4/18/22,4/19/22,4/20/22,4/21/22,4/22/22,4/23/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,178352,178373,178387,178418,178457,178513,178574,178611,178638,178648
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,274376,274429,274462,274504,274520,274535,274606,274606,274737,274791


Use pandas to create a dataframe that aggregates and sums both confirmed cases and deaths on a global level

#Transforming the Date format from 1/2/20 to YYYY-MM-DD (2020-01-02)
 

In [3]:
total_confirmed= confirmed_df.iloc[:, 4:].sum(axis=0)
total_death= death_df.iloc[:, 4:].sum(axis=0)

frame = { 'Covid Confirmed': total_confirmed, 'Covid Death': total_death }
aggregated_df = pd.DataFrame(frame )

as_list = aggregated_df.index.tolist()
new_list = []
for i in as_list:
    nums = i.split('/')
    month_leading_zero='0'
    if(int(nums[0]) > 9): month_leading_zero = ''

    day_leading_zero='0'
    if(int(nums[1]) > 9): day_leading_zero = ''

    new_list.append('20'+nums[2]+'-'+month_leading_zero+''+nums[0]+'-'+day_leading_zero+nums[1])

aggregated_df.index = new_list
aggregated_df = aggregated_df.transpose()
aggregated_df.head(5)

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2022-04-14,2022-04-15,2022-04-16,2022-04-17,2022-04-18,2022-04-19,2022-04-20,2022-04-21,2022-04-22,2022-04-23
Covid Confirmed,557,657,944,1437,2120,2929,5580,6169,8237,9927,...,502932802,503645731,504198041,504619899,505089291,506082451,507046645,507947338,508708884,509196481
Covid Death,17,18,26,42,56,82,131,133,171,213,...,6193401,6195670,6197182,6198487,6199914,6203317,6207815,6211981,6215610,6217046


Alpha Vantage! API key: 91MNBSNGPV50MFCO. 

In [5]:
!pip install alpha_vantage

In [6]:
from alpha_vantage.timeseries import TimeSeries
# ts = TimeSeries(key='91MNBSNGPV50MFCO')
# 
ts = TimeSeries(key='ED7TWFSOZVK81POS')
# Get json object with the intraday data and another with  the call's metadata


In [ ]:
# series = pd.read_json(data, typ='series', orient='records')
# print(series)
# Google - Googl - American
data1, meta_data1 = ts.get_daily(symbol='GOOGL',outputsize='full')
# Royal Bank Canada Aka RBC - TSE - Canadian
data2, meta_data2 = ts.get_daily(symbol='TSE',outputsize='full')
# AirBnb - ABNB - Travel Sector
data3, meta_data3 = ts.get_daily(symbol='ABNB',outputsize='full')
# Annaly Capital Management Inc. - NLY - Real estate
data4, meta_data4 = ts.get_daily(symbol='NLY',outputsize='full')
# Gold - AU - Metal
data5, meta_data5 = ts.get_daily(symbol='AU',outputsize='full')